In [ ]:
/// w ##class(web.DHCDocMain).SendInfo("160752","22027","11")
ClassMethod SendInfo(EpisodeID, UserId, LocId) [ Language = objectscript ]
{
	/*
	<RequestData>
	    <userid>00131</userid>
	    <deptid>1112101</deptid>
	    <patientType>1</patientType>
	    <patientId>0013008646</patientId>
	    <patientNo>256542222</patientNo>
	    <visit_id>4</visit_id>
	    <clinicDate>2023-10-01 12:22:33</clinicDate>
	    <patientName>于某某</patientName>
	    <certificateOfIdentification>370725196600000000</certificateOfIdentification>
	    <sex>1</sex>
	    <dateOfBirth>1980-01-01</dateOfBirth>
	    <phone>13500000000000</phone>
	    <nation>汉族</nation>
	    <address>xx省xx市xx县xx街道xx村11号</address>
	    <houseAddress>xx省xx市xx县xx街道xx村11号</houseAddress>
	    <occupation>农民</occupation>
	    <workOrg>xx单位</workOrg>
	    <marriage>已婚</marriage>
	    <diagnosisList>
	        <diagnosis>
	            <diagnosisCode>Ixxx</diagnosisCode>
	            <diagnosisName>手足口病</diagnosisName>
	            <diagnosisType>门诊诊断</diagnosisType>
	        </diagnosis>
	        <diagnosis>
	            <diagnosisCode>Ixxx</diagnosisCode>
	            <diagnosisName>上呼吸道感染</diagnosisName>
	            <diagnosisType>入院诊断</diagnosisType>
	        </diagnosis>
	    </diagnosisList>
	</RequestData>
	*/
	q:EpisodeID="" ""
	s ^sendinfo = $lb(EpisodeID,UserId,LocId)
	s AdmType = $p($g(^PAADM(EpisodeID)),"^",2)
	s patientType = $case(AdmType,"I":1,"O":2)
	s userid = $p($g(^SSU("SSUSR",UserId)),"^",1)
	s deptid = $p($g(^CTLOC(LocId)),"^",1)
	s PatientID = $p($g(^PAADM(EpisodeID)),"^",1)
	;病案号
	;门诊号
	s obj = ##class(User.PAPatMas).%OpenId(PatientID)
	
	s patientId = ##class(web.DHCDocOrderCommon).GetMrNo(EpisodeID,"",AdmType,"")	
	s patientNo = obj.PAPMIOPNo
	&sql(
		select count(1) into :visitId from SQLUser.pa_adm where PAADM_Type = :EpisodeID and PAADM_VisitStatus = "A" and PAADM_PAPMI_DR = :PatientID
	)
	s clinicDate = $zdt($h,3)
	s patientName = obj.PAPMIName
	s certificateOfIdentification = obj.PAPMIID
	s sex = obj.PAPMISexDR.%Id()
	s dateOfBirth = $zd(obj.PAPMIDOB,3)
	s phone = obj.PAPMIMobPhone
	
 	Set nation = $s(+$p($g(^PAPER(PatientID,"PER",2)),"^",1)'=0:$p($g(^CT("NAT",$p($g(^PAPER(PatientID,"PER",2)),"^",1))),"^",2),1:"")
	s address = $p($g(^PAPER(PatientID,"PER","ADD",1)),"^",1)
	s DHCPersonID=$O(^DHCPERSON(0,"PAPERSON",PatientID,0))
	s houseAddress=""
	s:DHCPersonID'="" houseAddress = $P(^DHCPERSON(DHCPersonID),"^",20)
	Set occupation = $s(+$p($g(^PAPER(PatientID,"PER",2)),"^",6)'=0:$p($g(^CT("OCC",$p($g(^PAPER(PatientID,"PER",2)),"^",6))),"^",2),1:"")
    set workOrg=$p($g(^PAPER(PatientID,"PER",4)),"^",18)
    set marital=$p($g(^PAPER(PatientID,"PER",2)),"^",3)
    &sql(select CTMAR_Desc into :marriage from SQLUser.CT_Marital where CTMAR_RowId = :marital)
    set writer=##class(%XML.Writer).%New()
    ;set writer.Indent=1
    ;set writer.Charset="UTF-8"

    d writer.OutputToString()
    d writer.StartDocument()
    
    d writer.RootElement("RequestData")
    	d writer.Element("userid")
        d writer.WriteChars(userid)
        d writer.EndElement()
        
        d writer.Element("deptid")
        d writer.WriteChars(deptid)
        d writer.EndElement()
        
        d writer.Element("patientType")
        d writer.WriteChars("1")
        d writer.EndElement()
        
        d writer.Element("patientId")
        d writer.WriteChars(patientId)
        d writer.EndElement()
        
        d writer.Element("patientNo")
        d writer.WriteChars(patientNo)
        d writer.EndElement()
        
        d writer.Element("visit_id")
        d writer.WriteChars(visitId)
        d writer.EndElement()
        
        d writer.Element("clinicDate")
        d writer.WriteChars(clinicDate)
        d writer.EndElement()
        
        d writer.Element("patientName")
        d writer.WriteChars(patientName)
        d writer.EndElement()
        
        d writer.Element("certificateOfIdentification")
        d writer.WriteChars(certificateOfIdentification)
        d writer.EndElement()
        
        d writer.Element("sex")
        d writer.WriteChars(sex)
        d writer.EndElement()
        
        d writer.Element("dateOfBirth")
        d writer.WriteChars(dateOfBirth)
        d writer.EndElement()
        
        d writer.Element("phone")
        d writer.WriteChars(phone)
        d writer.EndElement()
        
        d writer.Element("nation")
        d writer.WriteChars(nation)
        d writer.EndElement()
        
        d writer.Element("address")
        d writer.WriteChars(address)
        d writer.EndElement()
        
        d writer.Element("houseAddress")
        d writer.WriteChars(houseAddress)
        d writer.EndElement()
        
        d writer.Element("occupation")
        d writer.WriteChars(occupation)
        d writer.EndElement()
        
        d writer.Element("workOrg")
        d writer.WriteChars(workOrg)
        d writer.EndElement()
        
        d writer.Element("marriage")
        d writer.WriteChars(marriage)
        d writer.EndElement()
        
        d writer.Element("diagnosisList")
        	
        	&sql(select MRADM_RowId into :mradm from SQLUser.MR_Adm 
        		where MRADM_ADM_DR = :EpisodeID
        	)
        	s sub = 0
        	for {
	        	s sub=$O(^MR(mradm,"DIA",sub)) 
	        	Q:sub=""
	        	s diag = ##class(User.MRDiagnos).%OpenId($lts($lb(mradm,sub),"||"))
	        	d writer.Element("diagnosis")
		            d writer.Element("diagnosisCode")
		            s ICDRowid=$p(^MR(mradm,"DIA",sub),"^",1)
		            d writer.WriteChars($p($g(^MRC("ID",ICDRowid)),"^",1))
		            d writer.EndElement()
		            d writer.Element("diagnosisName")
			        d writer.WriteChars($p($g(^MRC("ID",ICDRowid)),"^",2))
			        d writer.EndElement()
			        d writer.Element("diagnosisType")
			        
			        s typeDesc = ""
			        s type = 0
			        for  {
				        s type=$o(^MR(mradm,"DIA",sub,"TYP",type)) 
				        Quit:type=""
					    s typeDr=$g(^MR(mradm,"DIA",sub,"TYP",type))
					   	s typeDesc=$p($g(^MRC("DTYP",typeDr)),"^",2)
					   	;q:typeDesc'=""
			        }
			        
			        ;s typeDesc=..GetDiagType($lts($lb(mradm,sub),"||"))
			        d writer.WriteChars(typeDesc)
			        d writer.EndElement()
	            d writer.EndElement()
	        }
            
           	
        d writer.EndElement()
    
    d writer.EndRootElement()
        
    d writer.EndDocument()
    
    s string=writer.GetXMLString()
    b ;;; string
    s out = ##class(%Stream.GlobalCharacter).%New()
    d out.Write(string)
    s xml = ##class(web.DHCENS.EnsWebService).GGWSReportCard(out)
    b ;;; ;xml
    
    Set reader = ##class(%XML.Reader).%New()
    
    Do reader.OpenString(xml)    //将XML字符串加载到读取器中
    s node = reader.Document.GetDocumentElement()    //获取根节点
    d node.GetText(.text)
    
    if (node.HasChildNodes()){
	    s text=""
	    d node.MoveToFirstChild(1)
	    d node.GetText(.text)
	    s status = text
    	While node.MoveToNextSibling(1) {
	    	s text=""
        	d node.GetText(.text)
        	s message = text
   		}
	}
    Do reader.Close()
    q $lts($lb(status,message),"!!")
    /*
    <OperationResult>
		<State>1</State>
		<Message>http://xxx/Home/Index?uname=xx&amp;patid=xx&amp;zycs=xx&amp;brlx=2&amp;t=4</Message>
	</OperationResult>
    */
}

In [ ]:
/// w ##class(web.DHCDocMain).HealthDoc(502883,19053,$lg(^HealthDoc,3),$lg(^HealthDoc,4))
ClassMethod HealthDoc(PatientID, UserID, timestamp, nonce) [ Language = objectscript ]
{
	
	/*
	https://yyzc.gdgov.cn/ebus/jiankangdangan/gdehr/dhccApi/codeAuthRec/save
	codeAuthRec 授权记录 Object
	authDays 有效期天数 int 固定 30 天
	patientIdcard 患者身份证号 String SM4 加密详见 9.3.3 加密代码示例
	patientName 患者姓名 String
	orgCode 医疗机构代码 String 系统下发
	orgName 机构名称 String 系统下发
	status 有效性 String 1 有效，0 无效
	appId 应用 ID String 系统下发
	*/
	/*
	param
	SM4 加密的查询参数，字符集使用 UTF-8，原值：
	orgCode={1}&staffNo={2}&staffName={3}&card
	Type={4}&idcard={5}&patientName={6}&patientI
	d={7}&callSysCode={8} String
	appId 调阅方应用 ID String 系统下发
	*/
	/*
	orgCode 机构代码 String 系统下发
	staffNo 医生账号 String 调阅系统的登录账号
	staffName 医生姓名 String
	cardType 患者证件类型 String 详见附件 B.1 证件类型代码
	idcard 患者证件号码 String
	patientName 患者姓名 String
	patientId 患者院内 ID String 传患者院内 ID
	callSysCode 集成方代码 String 系统下发
	*/
	/*
	19.15.82.240 yyzc.gdgov.cn
	19.15.86.130 qmjk.gdgov.cn
	*/
	w ##class(web.Util.SM4).Encrypt("f575543b530aea0d46b9d78a7e563c2b","124453814564938439","HEX","TEXT")
	q:PatientID="" ""
	q:UserID="" ""
	s ^HealthDoc = $lb(PatientID, UserID, timestamp, nonce)
	s UserID = 19463
	s appId = "ff8080818b66bbbc018e1b6dad65008d"
	s key = "f575543b530aea0d46b9d78a7e563c2b"
	s paasid = "luodingshirenminyiyuan"
	s paastoken = "Bearer 9d447f314fe743b1a8fa9b9fe799b873"
	s orgCode = "124453814564938439"
	s orgName = "罗定市人民医院"
	s callSysCode = "ldsrmyy"
	&sql(
		select PAPMI_CardType_DR->Card_Code ,PAPMI_DVAnumber, PAPMI_Name, PAPMI_RowId1 
			into :cardType, :idcard, :patientName, :patientId
		from SQLUser.PA_PatMas 
		where PAPMI_RowId1 = :PatientID
	)
	s orgCode = orgCode
	s staffNo = $p($g(^SSU("SSUSR",UserID)),"^",1)
	s staffName = $p($g(^SSU("SSUSR",UserID)),"^",2)
	s cardType = cardType
	s idcard = idcard
	s patientName = patientName
	s patientId = patientId
	
	
	// 授权接口
	s json = {}
	s codeAuthRec = {}
	s codeAuthRec.authDays = "30"
	
	SET tempFile = "/dthealth/db/dthis/data/"_idcard_".hex"

	set status = $ZF(-100,"/SHELL","echo","-n",idcard,"|","openssl","enc","-sm4-ecb","-K","f575543b530aea0d46b9d78a7e563c2b","|","xxd","-p",">",idcard_".hex")
	
	// 读取文件内容到变量
	set file = ##class(%File).%New(tempFile)
    set sc = file.Open("R")
    if $$$ISERR(sc) quit sc    ; or do smth

	s hex = ""
    while 'file.AtEnd {
        set str=file.ReadLine() 
        set hex = hex _ str
    }
    do file.Close()
	
	d ##class(%File).Delete(tempFile, .return)
	s hex = $ZCONVERT(hex,"U")
	// 输出捕获的内容
	d ##class(%File).Delete(tempFile, .return)
	b ;;;; hex
	
	s codeAuthRec.patientIdcard = hex
	s codeAuthRec.patientName = patientName
	s codeAuthRec.orgCode = orgCode
	s codeAuthRec.orgName = orgName
	s codeAuthRec.status = "1"
	s codeAuthRec.appId = appId
	s json.codeAuthRec = codeAuthRec
	;https://yyzc.gdgov.cn/ebus/jiankangdangan/gdehr/dhccApi/codeAuthRec/save
	/*
	curl "$url$path" -H "Content-Type:application/json" -H
	"x-tif-timestamp:$currentTimestap" -H "x-tif-paasid:$passid" -H
	"x-tif-nonce:$nonce" -H "x-tif-signature:$signsha" -X POST -d $body

	var timestamp = (Date.now() / 1000).toFixed();
	var nonce = Math.random().toString(36).substr(2);
	var paasid = "***";
	var paastoken = "***";
	var signature = CryptoJS.SHA256(timestamp + paastoken + nonce + timestam
	p).toString(CryptoJS.enc.Hex).toUpperCase();
	postman.setGlobalVariable("timestampHeader", timestamp);
	postman.setGlobalVariable("signatureHeader", signature);
	postman.setGlobalVariable("paasidHeader", paasid);
	postman.setGlobalVariable("nonceHeader", nonce);
	*/
	set request=##class(%Net.HttpRequest).%New()
	set request.Server="19.15.82.240"
	set request.ContentType="application/json"
	;set request.Https=1
	s signature = ##class(CA.Util.Encryption).SHA256AndHex(timestamp_paastoken_nonce_timestamp,1)
	;w ##class(CA.Util.Encryption).SHA256AndHex("44088219940814304X")
	;s status = request.SetHeader("Content-Type", "application/json")
	s ^HealthDoc("sign") = signature
	s status = request.SetHeader("x-tif-timestamp", timestamp)
	s status = request.SetHeader("x-tif-paasid", paasid)
	s status = request.SetHeader("x-tif-nonce", nonce)
	s status = request.SetHeader("x-tif-signature", signature)
	b ;;; header
	;Do request.Send("GET","url")
	;set request.SSLConfiguration="TEST"
	
	s status = request.EntityBody.Write(json.%ToJSON())
	set status=request.Post("/ebus/jiankangdangan/gdehr/dhccApi/codeAuthRec/save")
	Set response=request.HttpResponse.Data.Read()
	b ;;;;;; response
	s ^HealthDoc("res") = response
	;q response
	;###########################################################
	// 调阅接口
	s text = $lts(
		$lb(
			$lts($lb("orgCode",orgCode), "="),
			$lts($lb("staffNo",staffNo), "="),
			$lts($lb("staffName",staffName), "="),
			$lts($lb("cardType",cardType), "="),
			$lts($lb("idcard",idcard), "="),
			$lts($lb("patientName",patientName), "="),
			$lts($lb("patientId",patientId), "="),
			$lts($lb("callSysCode",callSysCode), "=")
		),
		"&"
	)
	s ^HealthDoc("param") = text
	;s param = ##class(web.Util.SM4).Encrypt(key,text,"HEX")
	// https://qmjk.gdgov.cn/gdehr/privilege/getPrivilege
	SET tempFile = idcard_".txt"
	set status = $ZF(-100,"/SHELL","echo","'"_text_"'",">",tempFile)
	set status = $ZF(-100,"/SHELL",
		"cat",tempFile,"|",
		"tr", "-d" ,"'\n'","|",
		"openssl","enc","-sm4-ecb","-K","f575543b530aea0d46b9d78a7e563c2b","|",
		"xxd","-p",
		">",idcard_".arh"
	)
	set status = $ZF(-100,"/SHELL","rm",tempFile)
	;d ##class(%File).Delete(tempFile, .return)
	// 读取文件内容到变量
	SET tempFile = "/dthealth/db/dthis/data/"_idcard_".arh"
	set file = ##class(%File).%New(tempFile)
    set sc = file.Open("R")
    if $$$ISERR(sc) quit sc    ; or do smth

	s hex = ""
    while 'file.AtEnd {
        set str=file.ReadLine() 
        set hex = hex _ str
    }
    do file.Close()
	s ^HealthDoc("secret") = hex
	set status = $ZF(-100,"/SHELL","rm",idcard_".arh")
	s hex = $ZCONVERT(hex,"U")

	set request=##class(%Net.HttpRequest).%New()
	set request.Server="19.15.86.130"
	;set request.Https=1
	;set request.SSLConfiguration="TEST"
	;s status = request.SetHeader("x-tif-timestamp", timestamp)
	;s status = request.SetHeader("x-tif-paasid", paasid)
	;s status = request.SetHeader("x-tif-nonce", nonce)
	;s status = request.SetHeader("x-tif-signature", signature)
	
	Do request.SetParam("param",hex)
	Do request.SetParam("appId",appId)
	
	;Do request.SetHeader("passid", passid)
	;Do request.SetHeader("passtoken", passtoken)
	;Do request.Send("GET","url")
	set status=request.Post("/gdehr/privilege/getPrivilege")
	Set response=request.HttpResponse.Data.Read()
	
	s loc = request.HttpResponse.GetHeader("LOCATION")
	s ^HealthDoc("loc") = loc
	s ^HealthDoc("url") = response
	// 解密
	q loc
}

In [ ]:
echo -n 445381199801084075 | openssl enc -sm4-ecb -K f575543b530aea0d46b9d78a7e563c2b | xxd -p > data.txt

https://docs.openeuler.org/en/docs/23.03/docs/ShangMi/algorithm-library.html

set status = $ZF(-100,"/SHELL","echo","-n","445381199801084075","|","openssl", "enc", "-sm4-ecb",  "-K","f575543b530aea0d46b9d78a7e563c2b","|","xxd","-p",">","data.txt")

In [ ]:
ClassMethod Nonce()
{
	SET charset = "abcdefghijklmnopqrstuvwxyz0123456789"
	SET nonce = ""
	FOR i=1:1:10 {
	    SET nonce = nonce_$EXTRACT(charset, $RANDOM($LENGTH(charset)) + 1)
	}
	q nonce
}

/// w ##class(web.DHCDocMain).HealthDoc("456924","18881")
ClassMethod HealthDoc(EpisodeID, UserID)
{
	/*
	19.15.82.240 yyzc.gdgov.cn
	19.15.86.130 qmjk.gdgov.cn
	*/
	q:EpisodeID="" ""
	q:UserID="" ""
	s ^HealthDoc = $lb(EpisodeID, UserID)
	s PatientID = $p($g(^PAADM(EpisodeID)),"^",1)
	s timestamp = $zdt($h,-2)
	s nonce = ..Nonce()
	;s:UserID="" UserID = 19463
	s Hosp = ##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeID)
	if Hosp = 2 {
		s appId = "ff8080818b66bbbc018e1b6dad65008d"
		s key = "f575543b530aea0d46b9d78a7e563c2b"
		s paasid = "luodingshirenminyiyuan"
		s paastoken = "Bearer 9d447f314fe743b1a8fa9b9fe799b873"
		s orgCode = "124453814564938439"
		s orgName = "罗定市人民医院"
		s callSysCode = "ldsrmyy"
	}
	if Hosp = 3 {
		s appId = "ff8080818b66bbbc018e196327d20089"
		s key = "544f4053391f2cc25994a4daacec6113"
		s paasid = "luodingshihongshizihuiyiyuan"
		s paastoken = "Bearer d53bb26fe27a47959230e0b8983d5efe"
		s orgCode = "124453814564953980"
		s orgName = "罗定市罗城医院"
		s callSysCode = "ldshszhyy"
	}
	if Hosp = 11 {
		s appId = "ff8080818b66bbbc018e1962152f0087"
		s key = "7cd1f2d74c6db172d4865b5d43364744"
		s paasid = "luodingshizhongyiyuan"
		s paastoken = "Bearer 25333a7335af4f62bd0d8dc299038260"
		s orgCode = "1244538145649378X8"
		s orgName = "罗定市中医院"
		s callSysCode = "ldszyy"
	}
	&sql(
		select PAPMI_CardType_DR->Card_Code ,PAPMI_DVAnumber, PAPMI_Name, PAPMI_RowId1 
			into :cardType, :idcard, :patientName, :patientId
		from SQLUser.PA_PatMas 
		where PAPMI_RowId1 = :PatientID
	)
	
	s orgCode = orgCode
	s staffNo = $p($g(^SSU("SSUSR",UserID)),"^",1)
	s staffName = $p($g(^SSU("SSUSR",UserID)),"^",2)
	s cardType = cardType
	s idcard = idcard
	s patientName = patientName
	s patientId = patientId
	
	// 调阅接口
	s text = $lts(
		$lb(
			$lts($lb("orgCode",orgCode), "="),
			$lts($lb("staffNo",staffNo), "="),
			$lts($lb("staffName",staffName), "="),
			$lts($lb("cardType",cardType), "="),
			$lts($lb("idcard",idcard), "="),
			$lts($lb("patientName",patientName), "="),
			$lts($lb("patientId",patientId), "="),
			$lts($lb("callSysCode",callSysCode), "=")
		),
		"&"
	)
	s ^HealthDoc("param")=text
	s param = ##class(web.Util.SM4).Encrypt(key,text,"HEX","HEX")
	
	set request=##class(%Net.HttpRequest).%New()
	set request.Server="19.15.86.130"
	Do request.SetParam("param",param)
	Do request.SetParam("appId",appId)
	/*
	set request.Https = 1
	set request.SSLConfiguration = "SSLECP"
	set request.SSLCheckServerIdentity = 0
	*/
	set status=request.Post("/gdehr/privilege/getPrivilege")
	Set response=request.HttpResponse.Data.Read()
	s loc = request.HttpResponse.GetHeader("LOCATION")
	
	s ^HealthDoc("loc") = loc
	q loc
}

/// w ##class(web.DHCDocMain).CallHealthDoc("456924","18881",2)
ClassMethod CallHealthDoc(PatNo, UserCode, Hosp)
{
	/*
	19.15.82.240 yyzc.gdgov.cn
	19.15.86.130 qmjk.gdgov.cn
	*/
	// 授权接口
	q:PatNo="" ""
	q:Hosp="" ""
	&sql(
		select SSUSR_RowId into :UserID
		from SQLUser.ss_user 
		where SSUSR_Initials = :UserCode
	)
	;q:UserID="" ""
	s ^HealthDoc = $lb(PatNo, UserID)
	s PatientID = $O(^PAPERi("PAPMI_PatNo",PatNo,0)) ;$p($g(^PAADM(EpisodeID)),"^",1)
	s timestamp = $zdt($h,-2)
	s nonce = ..Nonce()
	s:UserID="" UserID = 18881
	;s Hosp = ##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeID)
	if Hosp = 2 {
		s appId = "ff8080818b66bbbc018e1b6dad65008d"
		s key = "f575543b530aea0d46b9d78a7e563c2b"
		s paasid = "luodingshirenminyiyuan"
		s paastoken = "Bearer 9d447f314fe743b1a8fa9b9fe799b873"
		s orgCode = "124453814564938439"
		s orgName = "罗定市人民医院"
		s callSysCode = "ldsrmyy"
	}
	if Hosp = 3 {
		s appId = "ff8080818b66bbbc018e196327d20089"
		s key = "544f4053391f2cc25994a4daacec6113"
		s paasid = "luodingshihongshizihuiyiyuan"
		s paastoken = "Bearer d53bb26fe27a47959230e0b8983d5efe"
		s orgCode = "124453814564953980"
		s orgName = "罗定市罗城医院"
		s callSysCode = "ldshszhyy"
	}
	if Hosp = 11 {
		s appId = "ff8080818b66bbbc018e1962152f0087"
		s key = "7cd1f2d74c6db172d4865b5d43364744"
		s paasid = "luodingshizhongyiyuan"
		s paastoken = "Bearer 25333a7335af4f62bd0d8dc299038260"
		s orgCode = "1244538145649378X8"
		s orgName = "罗定市中医院"
		s callSysCode = "ldszyy"
	}
	&sql(
		select PAPMI_CardType_DR->Card_Code ,PAPMI_DVAnumber, PAPMI_Name, PAPMI_RowId1 
			into :cardType, :idcard, :patientName, :patientId
		from SQLUser.PA_PatMas 
		where PAPMI_RowId1 = :PatientID
	)
	
	s orgCode = orgCode
	s staffNo = $p($g(^SSU("SSUSR",UserID)),"^",1)
	s staffName = $p($g(^SSU("SSUSR",UserID)),"^",2)
	s cardType = cardType
	s idcard = idcard
	s patientName = patientName
	s patientId = patientId
	
	
	// 授权接口
	s card = ##class(web.Util.SM4).Encrypt(key,idcard,"HEX","HEX")
	s ^HealthDoc("id") = $lb(idcard,card)
	s json = {}
	s codeAuthRec = {}
	s codeAuthRec.authDays = 30
	s codeAuthRec.patientIdcard = card
	s codeAuthRec.patientName = patientName
	s codeAuthRec.orgCode = orgCode
	s codeAuthRec.orgName = orgName
	s codeAuthRec.status = 1
	s codeAuthRec.appId = appId
	s json.codeAuthRec = codeAuthRec
	
	set request=##class(%Net.HttpRequest).%New()
	set request.Server="19.15.82.240"
	set request.ContentType="application/json;charset=UTF-8"
	s sign = timestamp_paastoken_nonce_timestamp
	s signature = ##class(CA.Util.Encryption).SHA256AndHex(sign,1)
	s ^HealthDoc("sign") = $lb(sign,signature)
	s ^HealthDoc("json") = $lb(json.%ToJSON())
	s status = request.SetHeader("x-tif-timestamp", timestamp)
	s status = request.SetHeader("x-tif-paasid", paasid)
	s status = request.SetHeader("x-tif-nonce", nonce)
	s status = request.SetHeader("x-tif-signature", signature)
	s status = request.EntityBody.Write(json.%ToJSON())
	set status=request.Post("/ebus/jiankangdangan/gdehr/dhccApi/codeAuthRec/save")
	Set response=request.HttpResponse.Data.Read()
	b ;;;;
	s ^HealthDoc("res") = response
	s json = {}.%FromJSON(response)
	q json.returnCode
}


In [ ]:
/// w ##class(web.DHCDocMain).QueryURL("303566",21394,"2","")
ClassMethod QueryURL(EpisodeId, UserId)
{
	q:EpisodeId="" ""
	s UserId=21394
	s key = "043837054A1F83CDD7A456ECE82C238286E0604A17D4019ACB5E667DDFDF8F4DD1AECC9C049B880A8C1EC730F9B1EA9A1679E1CE6BBA8CF6E7AEB48D93BAD5D1D3"
	
	s ^QueryURL("input") = $lb(EpisodeId, UserId)
	s AdmType = $p($g(^PAADM(EpisodeId)),"^",2)
	s PatientID = $p($g(^PAADM(EpisodeId)),"^",1)
	
	s obj = ##class(User.PAPatMas).%OpenId(PatientID)
	q:obj="" ""
	s HospID=##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeId)
	
	s IdCard = obj.PAPMIID
	s Name = obj.PAPMIName
	s phone = obj.PAPMIMobPhone
	d obj.%Close()
	s user = ##class(User.SSUser).%OpenId(UserId)
	q:obj="" ""
	
	s DoctorName = user.SSUSRName
	s DoctorCode = user.SSUSRInitials
	s DoctorIdCard = user.SSUSRFreeText1
	
	s hospitalCode = "123704004933101887" 
	s hospitalName = "北京中医药大学东方医院枣庄医院"
	s timestamp = $zdt($h,-2)
	s json = {}
	s json.hospitalCode = hospitalCode
	
	s data = {}
	s data.ExamFlg = "2"
	s data.HospitalCode = hospitalCode
	s data.HospitalName = hospitalName
	s data.DoctorCode = DoctorCode
	s data.DoctorName = DoctorName
	s data.DoctorIdCard = DoctorIdCard
	s data.PatientId = PatientID
	s data.PatientName = Name
	s data.PatientIdCard = IdCard
	s data.PatientTelephone = "18899998888" ;phone
	s ^QueryURL("data") = data.%ToJSON()
	s hash = ##class(web.DHCENS.BLL.Encrypt.SMClass).encryptSM2Info(data.%ToJSON(),key)
	s json.data = hash
	s json.timestamp = timestamp
	
 	s signature = hash _ hospitalCode _ timestamp
 	s sign = ##class(web.DHCDocMain).SHA1(signature,IdCard)
 	s ^QueryURL("sign") = sign
	s json.signature = sign
	s ^QueryURL("json") = json.%ToJSON()
	set request=##class(%Net.HttpRequest).%New()
	set request.Server="report-api.zzyingxiangyun.com"
	;Do request.SetParam("param",param)
	;Do request.SetParam("appId",appId)
	set request.ContentType="application/json;charset=UTF-8"
	set request.Port=30080
	s status = request.EntityBody.Write(json.%ToJSON())
	set status=request.Post("/api/risexaminfo/check")
	
	Set response=request.HttpResponse.Data.Read()
	s ^QueryURL("res") = response
	;s rtn = ##class(web.DHCENS.EnsWebService).ApiRisExaminfoCheck(json.%ToJSON())
	;s rtn = ##class(web.DHCENS.EnsWebService).ApiMutualCheck(json.%ToJSON())
	
	s url = ""
	s res = {}.%FromJSON(response)
	if (res.code = 1) {
		if res.content > 0 {
			s url = res.message
		}	
	}
	s ^QueryURL("url") = url
	q url
}

/// w ##class(web.DHCDocMain).QueryURL("303566",21394,"2","")
ClassMethod QueryURLArc(EpisodeId, UserId, ArcimStr = "")
{
	q:EpisodeId="" ""
	s UserId=21394
	s key = "043837054A1F83CDD7A456ECE82C238286E0604A17D4019ACB5E667DDFDF8F4DD1AECC9C049B880A8C1EC730F9B1EA9A1679E1CE6BBA8CF6E7AEB48D93BAD5D1D3"
	
	s ^QueryURL("input") = $lb(EpisodeId, UserId, ArcimStr)
	s AdmType = $p($g(^PAADM(EpisodeId)),"^",2)
	s PatientID = $p($g(^PAADM(EpisodeId)),"^",1)
	
	s obj = ##class(User.PAPatMas).%OpenId(PatientID)
	q:obj="" ""
	s HospID=##class(DHCDoc.Common.Hospital).GetAdmHospitalId(EpisodeId)
	
	s IdCard = obj.PAPMIID
	s Name = obj.PAPMIName
	s phone = obj.PAPMIMobPhone
	d obj.%Close()
	s user = ##class(User.SSUser).%OpenId(UserId)
	q:obj="" ""
	
	s DoctorName = user.SSUSRName
	s DoctorCode = user.SSUSRInitials
	s DoctorIdCard = user.SSUSRFreeText1
	
	s hospitalCode = "123704004933101887" 
	s hospitalName = "北京中医药大学东方医院枣庄医院"
	s timestamp = $zdt($h,-2)
	s json = {}
	s json.hospitalCode = hospitalCode
	
	s data = {}
	s data.ExamFlg = "3"
	s data.HospitalCode = hospitalCode
	s data.HospitalName = hospitalName
	s data.DoctorCode = DoctorCode
	s data.DoctorName = DoctorName
	s data.DoctorIdCard = DoctorIdCard
	
	
	for i = 1:1:$l(ArcimStr,"$") {
		s arcim = $p(ArcimStr,"$",i)
		s code = $p(^ARCIM(+arcim,1,1),"^",1)
		s code=##class(web.API.JKXXPT.Method.Common).TransValue(code,"Health_ARCITM","HisCode",4)
		s $list(arr,i) = code
	}
	s data.MutualCode = $lts(arr,",")
	
	s data.PatientId = PatientID
	s data.PatientName = Name
	s data.PatientIdCard = IdCard
	s data.PatientTelephone = "18899998888" ;phone
	s ^QueryURL("data") = data.%ToJSON()
	s hash = ##class(web.DHCENS.BLL.Encrypt.SMClass).encryptSM2Info(data.%ToJSON(),key)
	s json.data = hash
	s json.timestamp = timestamp
	
 	s signature = hash _ hospitalCode _ timestamp
 	s sign = ##class(web.DHCDocMain).SHA1(signature,IdCard)
 	s ^QueryURL("sign") = sign
	s json.signature = sign
	s ^QueryURL("json") = json.%ToJSON()
	set request=##class(%Net.HttpRequest).%New()
	set request.Server="report-api.zzyingxiangyun.com"
	;Do request.SetParam("param",param)
	;Do request.SetParam("appId",appId)
	set request.ContentType="application/json;charset=UTF-8"
	set request.Port=30080
	s status = request.EntityBody.Write(json.%ToJSON())
	set status=request.Post("/api/mutual/check")
	
	Set response=request.HttpResponse.Data.Read()
	s ^QueryURL("res") = response
	;s rtn = ##class(web.DHCENS.EnsWebService).ApiMutualCheck(json.%ToJSON())

	s url = ""
	s res = {}.%FromJSON(response)
	if (res.code = 1) {
		if res.content = 1 {
			s url = res.message
		}	
	}
	s ^QueryURL("url") = url
	q url
}

/// w ##class(web.DHCDocMain).SHA1("姚鑫",123456)
ClassMethod SHA1(text, id)
{
	; 写入文件
	s home = "/dthealth/cachesys/mgr/"
	set tempFile = home_id_".new"
	set fs=##class(%Stream.FileCharacter).%New()
    set fs.Filename=tempFile
    set fs.TranslateTable="UTF8"
    do fs.Write(text)
    do fs.%Close()
    set sc=fs.%Save()
    ; bash
	set status = $ZF(-100,"/SHELL","bash","./user/sha1.sh",id_".new")
	s ^QueryURL("zf") = status_","_$USERNAME
	q:status'=0 text
	; 读取文件
	set tempFile = home_id_".out"
	set file = ##class(%File).%New(tempFile)
    set sc = file.Open("R")
    if $$$ISERR(sc) quit sc
	s hex = ""
    while 'file.AtEnd {
        set str=file.ReadLine() 
        set hex = hex _ str
    }
    do file.Close()
    ; 移除文件
	set status = $ZF(-100,"/SHELL","rm",id_".new",id_".out")
	s hex = $zcvt(hex,"U")
    q hex
}


In [ ]:
#!/bin/bash

# 检查是否提供了参数（文件名）
if [ $# -ne 1 ]; then
    echo "Usage: $0 <file-to-hash>"
    exit 1
fi

# 读取文件名参数
filename="$1"

# 检查文件是否存在
if [ ! -f "$filename" ]; then
    echo "Error: File '$filename' not found."
    exit 1
fi

# 计算输出文件名（原始文件名加上.out后缀）
output_filename="${filename%.*}.out"

# 使用openssl计算文件的SHA-1散列值，并将结果输出到新文件
# openssl sha1 "$filename" | tee "$output_filename"
cat "$filename" | tr -d '\n' | openssl sha1 | cut -d' ' -f2 > "$output_filename"

